**BIOSQUAD FINAL PROJECT NOTEBOOK**
<p> UN5550 Fall 2025</p>
<p> Lily Karg, Carly Steckling, Peter Briggs</p>

In [15]:
# Notebook Set Up

import xml.etree.ElementTree as ET
import pandas as pd
import requests

**Attempting to open an XML file for 2020https://raw.githubusercontent.com/lilykarg/biosquad/refs/heads/main/data/lccluc/2019.tif.aux.xml?token=GHSAT0AAAAAADO3WOCW67ASK6SKML74C6DG2IP7YKQ LCC data~!**
<p> 8 November 2025
<p> Lily

In [17]:
# Import LCC Files


# (Option to Import from local directory:)
# tree = ET.parse('____________.xml')
# root = tree.getroot()

# Import individual XML file from raw GitHub repo URL
url = "https://raw.githubusercontent.com/lilykarg/biosquad/refs/heads/main/data/lccluc/2019.tif.aux.xml?token=GHSAT0AAAAAADO3WOCW67ASK6SKML74C6DG2IP7YKQ"
response = requests.get(url)
xml_content = response.content
root = ET.fromstring(xml_content)
    
# Extract field names
field_names = []
for field in root.findall('.//GDALRasterAttributeTable/FieldDefn'):
    name = field.find('Name').text
    field_names.append(name)

# Extract row values
rows = []
for row in root.findall('.//GDALRasterAttributeTable/Row'):
    row_data = [f.text for f in row.findall('F')]
    rows.append(row_data)

# Create DataFrame
df = pd.DataFrame(rows, columns=field_names)

# Convert numeric columns to int
for col in df.columns:
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        pass  # non-numeric columns stay as strings

print(df.head())

   Pixel Value  Pixel Count  Red  Green  Blue  Opacity  \
0           11    460182049   70    107   159      255   
1         1112            0  162      1   255      255   
2         1121         8388  162      1   255      255   
3         1122        86608  162      1   255      255   
4         1123        30050  162      1   255      255   

                       NLCD Land Cover Class  
0                                 Open Water  
1           Open Water to Perennial Ice/Snow  
2        Open Water to Developed, Open Space  
3     Open Water to Developed, Low Intensity  
4  Open Water to Developed, Medium Intensity  


In [16]:
# Function for importing all XML LCC data files ----

def parse_gdal_xml(url):
    # Download XML content
    response = requests.get(url)
    root = ET.fromstring(response.content)
    
    # Extract field names
    field_names = [field.find('Name').text for field in root.findall('.//GDALRasterAttributeTable/FieldDefn')]
    
    # Extract rows
    rows = []
    for row in root.findall('.//GDALRasterAttributeTable/Row'):
        row_data = [f.text for f in row.findall('F')]
        rows.append(row_data)
    
    # Create DataFrame
    lcc_df = pd.DataFrame(rows, columns=field_names)
    
    # Convert numeric columns
    for col in lcc_df.columns:
        try:
            lcc_df[col] = pd.to_numeric(lcc_df[col])
        except ValueError:
            pass
    
    return lcc_df

In [23]:
# Import and combine all XML LCC data files ----
# this might pose a problem because it doesn't pull the year information with the files...!

# List raw data URLs:
urls = [
    "https://raw.githubusercontent.com/lilykarg/biosquad/refs/heads/main/data/lccluc/2019.tif.aux.xml?token=GHSAT0AAAAAADO3WOCW67ASK6SKML74C6DG2IP7YKQ",
    "https://raw.githubusercontent.com/lilykarg/biosquad/refs/heads/main/data/lccluc/2020.tif.aux.xml?token=GHSAT0AAAAAADO3WOCXI3JYOOZIQJYRIZTY2IP7ZBA",
]

# Append to master dataframe
all_lcc_dfs = []
for url in urls:
    lcc_df = parse_gdal_xml(url)
    all_lcc_dfs.append(lcc_df)

combined_lcc_df = pd.concat(all_lcc_dfs, ignore_index=True) # stack files vertically

# print(combined_lcc_df.head())
combined_lcc_df

,Pixel Value,Pixel Count,Red,Green,Blue,Opacity,NLCD Land Cover Class
0,11,460182049,70,107,159,255,Open Water
1,1112,0,162,1,255,255,Open Water to Perennial Ice/Snow
2,1121,8388,162,1,255,255,"Open Water to Developed, Open Space"
3,1122,86608,162,1,255,255,"Open Water to Developed, Low Intensity"
4,1123,30050,162,1,255,255,"Open Water to Developed, Medium Intensity"
...,...,...,...,...,...,...,...
507,9552,59840,162,1,255,255,Emergent Herbaceous Wetlands to Shrub/Scrub
508,9571,111746,162,1,255,255,Emergent Herbaceous Wetlands to Grassland/Herb...
509,9581,71633,162,1,255,255,Emergent Herbaceous Wetlands to Pasture/Hay
510,9582,134457,162,1,255,255,Emergent Herbaceous Wetlands to Cultivated Crops


**I'm not sure if this is how the data should look, but at least I was able to load it!!**
<p> big W